# Library

In [ ]:
import pandas as pd
import numpy as np
import os

from numpy import arange
import openpyxl
from io import BytesIO

 
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline


import itertools
import statsmodels.api as sm


from sklearn.metrics import r2_score


from tqdm import tqdm 
tqdm.pandas()




import statsmodels.api as sm
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX,SARIMAXResults
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from statsmodels.tools.eval_measures import rmse
from statsmodels.stats.stattools import durbin_watson


import matplotlib.pyplot as plt
import seaborn as sns




from feature_engine.transformation import ReciprocalTransformer
from sklearn.preprocessing import StandardScaler ,RobustScaler, QuantileTransformer ,PowerTransformer,MaxAbsScaler,MinMaxScaler,Normalizer
from sklearn.linear_model import LinearRegression,Ridge,RidgeCV,LarsCV,ridge_regression
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_absolute_percentage_error as MAPE,mean_squared_error

from scipy.stats import zscore,iqr
from scipy.stats.mstats import winsorize


import streamlit as st


from joblib import Parallel, delayed
import warnings

warnings.filterwarnings("ignore")


# Folder and File Path

In [ ]:
# Folder Path

Folder_path=r"D:\Pranto\Working\Forward Looking PD\Mar26 Prediction\V2\Data\SCV"

# file Name

File_name=r"SCV Raw Data 01042025 V2"


#Sheet Name


# sheet_name="None"
sheet_name="SCV_Retail_BKT1"



# Export file Name 

name=r"SCV Retail BKT1 Mean"


#################################################################################
##################################################################################

# # Test File Path

# test_path=rf"{Folder_path}\Independent Variables_Train Test.xlsx"



# # Test Data

# df_test=pd.read_excel(test_path,sheet_name="test")


# print(df_test.columns)

# df_test


#################################################################################
##################################################################################

# Import  Train Data

In [ ]:

# Read Data
if sheet_name=="None":
    df=pd.read_excel(rf"{Folder_path}\{File_name}.xlsx")

else:
    df=pd.read_excel(rf"{Folder_path}\{File_name}.xlsx",sheet_name=sheet_name)

df_date=df['Date']

print("Shape of the dataset",df.shape)
df.head(3)

# Create Log Variable for taking Log as dependent variable

In [ ]:
# Provide column name for creating log variable
create_log='PD_BKT_1'
df[create_log+"_Log(10)"]=np.log10(df[create_log])  # Create Log column

df.head(5)

# Filter on date

In [ ]:
# # Filter on date
# df1=df
 
# df1['Date']=pd.to_datetime(df1['Date'])
# df2=df1[df1['Date'] >= pd.to_datetime('2022-01-01')] #yyyy-mm-dd
 
 
# df=df2
# df.head(3)

In [ ]:

df.columns

# Input Section

### Change values in below section

In [ ]:
# Choose Dependent And Independent Variables


# Dependent Variables
# col_dependent='PD_BKT_1'
# col_dependent='PD_BKT_0_Near'
col_dependent=create_log+"_Log(10)"


#################################################################################
##################################################################################

num=2          # Z score value
percentile=0.95  # Percentile


#################################################################################
##################################################################################

# Select Below Outlier treatment Method

# method="None"
method="Mean Standard Deviation method"
# method="IQR Method"
# method="Percentile Method"


#################################################################################
##################################################################################

# Log and Lag variable Creation


lags=[1,2] # Lag value

ma_windows = [2,3] # For Moving Average


#################################################################################
##################################################################################

# 1st Standarization of Independent Variables


method_std='Standard Scaler'
# method_std='Log Transformation'
# method_std='Yeo-Johnson'
# method_std='Robust Scaler'
# method_std='Quantile Transformer'
# method_std='Max Abs Scaler'
# method_std='MinMax Scaler'
# method_std='Box-Cox Transformer'
# method_std='Reciprocal Transformer'
# method_std='Sin Transformation'
# method_std='Cos transformation'
# method_std='Tan Transformation'



#################################################################################
##################################################################################


# 2nd Standarization of Independent Variables

method_std_2nd="None"
# method_std_2nd='Standard Scaler'



#################################################################################
##################################################################################

# Others Parameter


adj_r2_num=0.5 # For Adjusted R2

vif_limit=5 # Vif Limit

dw_upper=2.5 # Durbin Watson Upper limit

dw_lower=1.5  # Durbin Watson Lower limit


p_val=0.054

comb_lower=2 # Lower Combination 
comb_upper=4 # Upper Combination


In [ ]:

###############################################################################
##################################################################################
# Choose Dependent And Independent Variables


# # Dependent Variables
# col_dependent='PD_BKT_2'
# # col_dependent='PD_BKT_0_Near'
# # col_dependent=create_log+"_Log(10)"



# Independent Variables

col_independent=['Electricity ',
                'GDP',
                'Manufacturing ',
                'Mining ',
                'RR',
                'CPI',
                'IIP',
                'IIP (YOY)',
                'Brent',
                'PMI',
                'WPI',
                'Bank_Rate',
                'Bank_Rate/RR',
                'Diesel',
                'Sales_Commerical_Vehicle',
                'Sales_Passenger_Vehicles',
                'Seasonality',
                'Quarter',
                'Construction (% of GDP)',
                'Consumer prices (% change pa; av)',
                'Current-account balance (% of GDP)', 
                'Domestic credit growth (%)',
                'Exports of G&S (% of GDP)', 'GDP deflator (% change; av)',
                'Gross fixed investment (% of GDP)',
                'Gross national savings/investment (%)', 
                'Imports of G&S (% of GDP)',
                'Lending interest rate (%)',
                'Long-term bond yield (%)',
                'Nominal GDP (LCU)', 'Policy interest rate (%)',
                'Private consumption (% of GDP)',
                'Private consumption (real % change pa)',
                'Private consumption, contribution to real GDP growth (% points)',
                'Real effective exchange rate (index; 2010=100)',
                'Real GDP (% change pa)', 'Real GDP (LCU)',
                'Total factor productivity growth (%)', 'Unemployment rate (%)',
                'Housing Price Index (% change)'
]


print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("\n")


print("Dependent variable = ",col_dependent)


print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("\n")

################################################################################
##################################################################################

# Outlier Treatement

col_dep=col_dependent # Column's name from you want to remove or replace outliers 
# num=2          # Z score value
# percentile=0.95  # Percentile


print(f"Outlier Treatement in Column {col_dep}")


print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("\n")


################################################################################
##################################################################################
# # Select Below Method

# # method="None"
# method="Mean Standard Deviation method"
# # method="IQR Method"
# # method="Percentile Method"


print(f"Seleceted Outlier Method = ",method)


print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("\n")

##################################################################################
##################################################################################

# Log and Lag variable Creation


# lags=[1,2,3] # Lag value

# ma_windows = [2,3] # For Moving Average

col_log=col_dependent  # Log Column

col_lag=col_dependent  # Lag Column

col_ma=col_dependent  # for Moving Average column


print(f"Selected column {col_lag}_Lag for Lag and {col_log}_Log(10) for creating Log Variable  and number of Lag = {lags} and Moving Average = {ma_windows}")


print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("\n")

##################################################################################
##################################################################################

# Remove Number of Rows from Last for test data

number=1


print(f"Remove last {number} of rows from last for test data")


print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("\n")

#################################################################################
##################################################################################

# 1st Standarization of Independent Variables


# method_std='Standard Scaler'
# method_std='Log Transformation'
# method_std='Yeo-Johnson'
# method_std='Robust Scaler'
# method_std='Quantile Transformer'
# method_std='Max Abs Scaler'
# method_std='MinMax Scaler'
# method_std='Box-Cox Transformer'
# method_std='Reciprocal Transformer'
# method_std='Sin Transformation'
# method_std='Cos transformation'
# method_std='Tan Transformation'



col=col_independent


# 2nd Standarization of Independent Variables

# method_std_2nd="None"
# method_std_2nd='Standard Scaler'
# method_std='Robust Scaler'
# method_std='Max Abs Scaler'
# method_std='MinMax Scaler'





print(f"Selected 1st standarization method : {method_std} and  \n2nd Standirization method : {method_std_2nd}")

# print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")



###############################################################################
##################################################################################

# Conditions For Possible Combinations

# Dependent Variable

Y1=col_dependent

# Y1=col_log+"_Log(10)"




# # Others Parameter


# adj_r2_num=0.4 # For Adjusted R2

# vif_limit=5 # Vif Limit

# dw_upper=2.5 # Durbin Watson Upper limit

# dw_lower=1.5  # Durbin Watson Lower limit


# p_val=0.05

# comb_lower=2 # Lower Combination 
# comb_upper=4 # Upper Combination


comb_upper=comb_upper+1

# df_itteration=df

delete_row=number  # Delete row from last for testing


################################################################################
##################################################################################

desired_directions={
    'Electricity ':'Negative',
    'GDP':'Negative',
    'Manufacturing ':'Negative',
    'Mining ':'Negative',
    'RR': 'Positive',
    'CPI': 'Positive',
    'IIP':'Negative',
    'IIP (YOY)':'Negative',
    'Brent': 'Positive',
    'PMI':'Negative',
    'WPI': 'Positive',
    'Bank_Rate': 'Positive',
    'Bank_Rate/RR': 'Positive',
    'Diesel': 'Positive',
    'Sales_Commerical_Vehicle':'Negative',
    'Sales_Passenger_Vehicles':'Negative',
    'Seasonality':'Both',
    'Quarter':'Both',
    'Construction (% of GDP)' : 'Negative',
    'Consumer prices (% change pa; av)' : 'Positive',
    'Current-account balance (% of GDP)' : 'Negative',
    'Domestic credit growth (%)' : 'Positive',
    'Exports of G&S (% of GDP)' : 'Negative',
    'GDP deflator (% change; av)' : 'Positive',
    'Gross fixed investment (% of GDP)' : 'Negative',
    'Gross national savings/investment (%)' : 'Negative',
    'Imports of G&S (% of GDP)' : 'Negative',
    'Lending interest rate (%)' : 'Positive',
    'Long-term bond yield (%)' : 'Positive',
    'Nominal GDP (LCU)' : 'Negative',
    'Policy interest rate (%)' : 'Positive',
    'Private consumption (% of GDP)' : 'Negative',
    'Private consumption (real % change pa)' : 'Negative',
    'Private consumption, contribution to real GDP growth (% points)' : 'Negative',
    'Real effective exchange rate (index; 2010=100)' : 'Positive',
    'Real GDP (% change pa)' : 'Negative',
    'Real GDP (LCU)' : 'Negative',
    'Total factor productivity growth (%)' : 'Negative',
    'Unemployment rate (%)' : 'Positive',
    'Housing Price Index (% change)' : 'Negative'
    
 
}




# Outliers Treatment

In [ ]:


# By Mean Standard Deviation method

if method=="Mean Standard Deviation method":

    df00=df
    df00=df00.drop('Date',axis=1)


    upper_limit0= df00[col_dep].mean() + num*df00[col_dep].std()
    lower_limit0 = df00[col_dep].mean() - num*df00[col_dep].std()

    Q1 = df00[col_dep].quantile(q=round((1-percentile),2))
    Q4 = df00[col_dep].quantile(q=percentile)

    # upper_limit0.values

    upper_limit=min(upper_limit0,Q4)
    lower_limit=max(lower_limit0,Q1)

    # upper_limit=min(float(upper_limit0),float(Q4))
    # lower_limit=max(float(lower_limit0[col]),float(Q1))

    df00_nonzero=df00[df00[col_dep]!=0]
    mean=np.mean(df00_nonzero)
    median=df00_nonzero.median()


    df00[col_dep]=df00[col_dep].replace(0,mean)

    lower_outlier=df00[df00[col_dep]< lower_limit]
    upper_outlier=df00[df00[col_dep]> upper_limit]


    loc_lower_outlier = lower_outlier[col_dep][lower_outlier[col_dep].notna()]
    loc_upper_outlier = upper_outlier[col_dep][upper_outlier[col_dep].notna()]

    # a=upper_outlier[upper_outlier[col].isna()].index.to_list()
    # a
    # print("**Lower Limit and Location of Outliers**")
    print("**Lower Limit :**",lower_limit0,"**Lower Quantile**",Q1,"**Final Lower Limit**",lower_limit)
    print("**Number of Lower Outliers :**",loc_lower_outlier.notna().sum())
    print("**Lower Outlier Location**",loc_lower_outlier)

    print("**Upper Limit :**",upper_limit0,"**Upper Quantile**",Q4,"**Final Upper Limit**",upper_limit)
    # Q4
    print("**Number of Upper Outliers :**",loc_upper_outlier.notna().sum())
    print("**Upper Outlier Location**",loc_upper_outlier)



    print("Replace Outliers by Percntile_Upper_Lower Limit")

    df00[col_dep]=df00[col_dep].replace(0,mean)

    df00[col_dep] = np.where(df00[col_dep]> upper_limit0, upper_limit,
    np.where(df00[col_dep]< lower_limit0, lower_limit,
        df00[col_dep]))

    df=df00

    print(df.shape)
    df.head()


if method=="Percentile Method":

    df00=df
    df00=df00.drop('Date',axis=1)

    df00_nonzero=df00[df00[col_dep]!=0]
    mean=np.mean(df00_nonzero)
    median=df00_nonzero.median()

    Q1 = df00[col_dep].quantile(q=round((1-percentile),2))
    Q4 = df00[col_dep].quantile(q=percentile)

    
    lower_limit0=Q1
    upper_limit0=Q4

    upper_limit=Q4
    lower_limit=Q1

    # df00_nonzero=df00[df00[col_dep]!=0]
    # mean=np.mean(df00_nonzero)
    # median=df00_nonzero.median()



    lower_outlier=df00[df00[col_dep]< lower_limit0]
    upper_outlier=df00[df00[col_dep]> upper_limit0]


    loc_lower_outlier = lower_outlier[col_dep][lower_outlier[col_dep].notna()]
    loc_upper_outlier = upper_outlier[col_dep][upper_outlier[col_dep].notna()]

    # a=upper_outlier[upper_outlier[col].isna()].index.to_list()
    # a
    # print("**Lower Limit and Location of Outliers**")
    print("**Lower Limit :**",lower_limit0,"**Lower Quantile**",Q1,"**Final Lower Limit**",lower_limit)
    print("**Number of Lower Outliers :**",loc_lower_outlier.notna().sum())
    print("**Lower Outlier Location**",loc_lower_outlier)

    print("**Upper Limit :**",upper_limit0,"**Upper Quantile**",Q4,"**Final Upper Limit**",upper_limit)
    # Q4
    print("**Number of Upper Outliers :**",loc_upper_outlier.notna().sum())
    print("**Upper Outlier Location**",loc_upper_outlier)



    print("Replace Outliers by Percntile_Upper_Lower Limit")

    df00[col_dep]=df00[col_dep].replace(0,mean)

    df00[col_dep] = np.where(df00[col_dep]> upper_limit0, upper_limit,
    np.where(df00[col_dep]< lower_limit0, lower_limit,
        df00[col_dep]))

    df=df00

    print(df.shape)
    df.head()


if method=="IQR Method":

    df00=df
    df00=df.drop('Date',axis=1)

    df00_nonzero=df00[df00[col_dep]!=0]
    mean=np.mean(df00_nonzero[col_dep])
    median=np.median(df00_nonzero)
    #find Q1, Q3, and interquartile range for each column
    Q1 = df00[col_dep].quantile(q=0.25)
    Q3 = df00[col_dep].quantile(q=0.75)
    # IQR = df00[col].apply(iqr)
    IQR=Q3-Q1

    lower_limit=Q1-1.5*IQR
    upper_limit=Q3+1.5*IQR

    
    lower_outlier=df00[df00[col_dep]< lower_limit]
    upper_outlier=df00[df00[col_dep]> upper_limit]


    loc_lower_outlier = lower_outlier[col_dep][lower_outlier[col_dep].notna()]
    loc_upper_outlier = upper_outlier[col_dep][upper_outlier[col_dep].notna()]

    
    print("**Lower Limit :**",lower_limit)
    print("**Number of Lower Outliers :**",loc_lower_outlier.notna().sum())
    print("**Lower Outlier Location**",loc_lower_outlier)

    print("**Upper Limit :**",upper_limit)
    print("**Number of Upper Outliers :**",loc_upper_outlier.notna().sum())
    print("**Upper Outlier Location**",loc_upper_outlier)
    
    

    print("**Want to replace Outlier**")

    df00[col_dep]=df00[col_dep].replace(0,mean)

    df00[col_dep] = np.where(df00[col_dep]> upper_limit, upper_limit,
    np.where(df00[col_dep]< lower_limit, lower_limit,
        df00[col_dep]))
    

    
    df=df00

    print(df.shape)

    df.head()
    



# Log,Lag,MA(Moving Average),EWMA(Exponential Weighted Moving Average) variable

In [ ]:
#  🔹 Create Log Variabl
df[col_log+"_Log(10)"]=np.log10(df[col_log])  # Create Log column

# 🔹 Create Lag Variables

for lag in lags:
    df[f'{col_lag}_Lag{lag}'] = df[col_lag].shift(lag)


# 🔹  Simple Moving Averages (MA)

for window in ma_windows:
    df[f'{col_ma}_{window}MA'] = df[col_ma].rolling(window=window).mean()
    df[f'{col_ma}_{window}SMA']=df[f'{col_ma}_{window}MA'].shift(1)

# 🔹  Exponential Moving Averages (EMA)

for window in ma_windows:
    df[f'{col_ma}_{window}EMA'] = df[col_ma].shift(1).ewm(span=window, adjust=False).mean()


# 🔹 Exponentially Weighted Moving Averages (EWMA)
# ewma_windows = [2, 4]
# for window in ewma_windows:
#     df[f'PD_{window}EWMA'] = df['PD'].ewm(com=window).mean()


###############################################################################
# 🔹 Determine the Maximum Window Size
max_lag = max(lags)  # Maximum lag used
max_ma = max(ma_windows)  # Maximum MA window

# Number of rows to remove
rows_to_drop = max(max_lag, max_ma)  # Find the max across all 


#[Because for each lag value the corresponding row will be blanks and for each Ma value (n-1) row would be blanks]


# 🔹  Remove rows from the HEAD only for maximum of (max(lag),max(ma_windows)-1)
df = df.iloc[rows_to_drop:]

# .reset_index(drop=True)
# Column list
lag_cols=[f'{col_lag}_Lag{lag}' for lag in lags]
sma_cols = [f'{col_ma}_{ma}SMA' for ma in ma_windows]
ema_cols = [f'{col_ma}_{ema}EMA' for ema in ma_windows]

new_columns=lag_cols+sma_cols+ema_cols


print("New Columns :\n",new_columns)
print(f"Delete {rows_to_drop} from the top of the dataset\n")                
print("Dataset Shape",df.shape)
df.tail(6)

                    

# Delete Rows From last

In [ ]:
# Delete number of rows from last




df0=df.tail(int(number))
df=df.head(-int(number))


# Remove Rows from Date column also according to delete 1st nth row because of MA,EMA,LAg

df_date1=df_date.iloc[rows_to_drop:]





print("Remove rows shape",df0.shape)

print("Rest Data shape",df.shape)

df.head()

# Dependent And Independent Variable

In [ ]:
df.columns

In [ ]:
# Dependent Variables

# independent columns with newly created lag,ma,ema columns
col_independent=col_independent+new_columns

col=col_independent

# Dependent 
y=df[col_dependent]


# Independent Variables
x=col_independent

# Independent Dataframe
x=df[x]

#Test data
x_test=df0[col_independent]


print("Independ Variable Dataset Shape",x.shape)
print("Dependent Variable Dataframe Shape",y.shape)
print("Test Dataframe Shape",x_test.shape)

# 1st Standarization

In [ ]:
if method_std=='Log Transformation':
    x[col]=np.log(x[col])
    x_test[col]=np.log(x_test[col])
    
    print(x.shape)

    # print(x.describe().T)


if method_std=='Standard Scaler':
    std_scaler=StandardScaler()
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    # x=std_scaler.fit_transform(x[col])
    # x=pd.DataFrame(x,columns=col)
    # x_test=std_scaler.transform(x_test[col])
    # x_test=pd.DataFrame(x_test,columns=col)

    
        
    print(x.shape)

    # Calculate mean and std for each column
    mean_values = std_scaler.mean_
    std_values = std_scaler.scale_

    # Print the mean and std values in tabular form
    summary_df = pd.DataFrame({
        'Column': col,
        'Mean': mean_values,
        'Std': std_values
    })
    
    print("Mean and Std of each column after scaling:")
    print(summary_df.head(5))

    


if method_std=='Normalizer':
    std_scaler=Normalizer()
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    
    
    print(x.shape)

    
    


if method_std=='Robust Scaler':
    std_scaler=RobustScaler()
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    print(x.shape)

    
    


if method_std=='Quantile Transformer':
    std_scaler=QuantileTransformer()
    x[col]=std_scaler.fit_transform(x[col])

   
    x_test[col]=std_scaler.transform(x_test[col])

    print(x.shape)

    

if method_std=='Max Abs Scaler':
    std_scaler=MaxAbsScaler()
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    print(x.shape)

    


if method_std=='MinMax Scaler':
    std_scaler=MinMaxScaler()
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    print(x.shape)

    


if method_std=='Box-Cox Transformer':
    std_scaler=PowerTransformer(method='box-cox')
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    print(x.shape)

    


if method_std=='Yeo-Johnson':
    std_scaler=PowerTransformer(method='yeo-johnson')
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    print(x.shape)

    

if method_std=='Reciprocal Transformer':
    x[col]=1/x[col]
    x_test[col]=1/x_test[col]

    print(x.shape)

    
    # pass
if method_std=='Sin Transformation':
    x[col]=np.sin(x[col])
    x_test[col]=np.sin(x_test[col])

    print(x.shape)

    
    # pass
if method_std=='Cos transformation':
    x[col]=np.cos(x[col])
    x_test[col]=np.cos(x_test[col])

    print(x.shape)

    
    # pass
if method_std=='Tan Transformation':
    x[col]=np.tan(x[col])
    x_test[col]=np.tan(x_test[col])

    print(x.shape)

    
            

# 2nd Standarization

In [ ]:
# method_std_2nd="None"
# # method_std_2nd='Standard Scaler'
# # method_std='Robust Scaler'
# # method_std='Max Abs Scaler'
# # method_std='MinMax Scaler'

if method_std_2nd=="None":
    pass



if method_std_2nd=='Standard Scaler':
    std_scaler=StandardScaler()
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    # x=std_scaler.fit_transform(x[col])
    # x=pd.DataFrame(x,columns=col)
    # x_test=std_scaler.transform(x_test[col])
    # x_test=pd.DataFrame(x_test,columns=col)

    
        
    print(x.shape)

    # Calculate mean and std for each column
    mean_values = std_scaler.mean_
    std_values = std_scaler.scale_

    # Print the mean and std values in tabular form
    summary_df = pd.DataFrame({
        'Column': col,
        'Mean': mean_values,
        'Std': std_values
    })
    
    print("Mean and Std of each column after scaling:")
    print(summary_df.head(5))

    


if method_std_2nd=='Normalizer':
    std_scaler=Normalizer()
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    
    
    print(x.shape)

    
    


if method_std_2nd=='Robust Scaler':
    std_scaler=RobustScaler()
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    print(x.shape)

    

    

if method_std_2nd=='Max Abs Scaler':
    std_scaler=MaxAbsScaler()
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    print(x.shape)

    


if method_std_2nd=='MinMax Scaler':
    std_scaler=MinMaxScaler()
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    print(x.shape)

 
            

#  Model Data

In [ ]:
from datetime import datetime

x1=pd.concat([x,x_test],axis=0)

# df_date=df_date.dt.strftime("%b/%y")

model_data=pd.concat([df_date,x1,y],axis=1)



# Drop rows with missing values except in the 'Date' column
model_data = model_data.dropna(subset=[col for col in model_data.columns if col != 'Date'],how='all')



print(model_data.shape)
model_data.tail(6)


In [ ]:
len(y)
# x1

# Export Model Data

In [ ]:

# model_data.to_excel(rf"{Folder_path}\{File_name} Model Data.xlsx",sheet_name=f"{File_name}",index=False)



# Import Data

In [ ]:
# df = pd.read_excel(r"C:\Users\524264\Downloads\OUTLIER_REPLACED_SCV_FTU_BKT_0.xlsx")
# df=df.tail(-int(1))


df=model_data


# Delete 1st 1 columns Date

df=df.iloc[:,1:]
# df=df.iloc[:,1:]


print(df.shape)
df.tail(6)
# df.tail()
# DATA.shape

# Change the Dependent Variable and Other Parameters

In [ ]:

# # Dependent Variable

# Y1='PD_BKT1'




# # Others Parameter


# adj_r2_num=0.6 # For Adjusted R2

# vif_limit=5 # Vif Limit

# dw_upper=2.5 # Durbin Watson Upper limit

# dw_lower=1.5  # Durbin Watson Lower limit


# p_val=0.05

# comb_lower=2 # Lower Combination 
# comb_upper=4 # Upper Combination


# comb_upper=comb_upper+1

# # df_itteration=df

# delete_row=1  # Delete row from last for testing



# desired_directions={
#     'Electricity ':'Negative',
#     'GDP':'Negative',
#     'Manufacturing ':'Negative',
#     'Mining ':'Negative',
#     'RR': 'Positive',
#     'CPI': 'Positive',
#     'IIP':'Negative',
#     'IIP (YOY)':'Negative',
#     'Brent': 'Positive',
#     'PMI':'Negative',
#     'WPI': 'Positive',
#     'Bank_Rate': 'Positive',
#     'Bank_Rate/RR': 'Positive',
#     'Diesel': 'Positive',
#     'Sales_Commerical_Vehicle':'Negative',
#     'Sales_Passenger_Vehicles':'Negative',
#     'Seasonality':'Both',
#     'Quarter':'Both'
# }






In [ ]:
df0=df.tail(int(delete_row))

df_itteration=df.head(-int(delete_row))


print("Train Data Shape",df_itteration.shape)

print("Test Data Shape",df0.shape)

df_itteration.tail()

# Input Section

In [ ]:
# Column list
lag_cols=[f'{col_lag}_Lag{lag}' for lag in lags]
ma_cols = [f'{col_ma}_{ma}EMA' for ma in ma_windows]
sma_cols = [f'{col_ma}_{ema}SMA' for ema in ma_windows]

In [ ]:
# Create a DataFrame to store all the information
data = {
    "Input Section": [
        "Dependent Variable", 
        "Independent Variables", 
        "Outlier Method",
        "Outliers information",
        "percentile",
        "Lag and Log Variable",
        "SMA Columns",
        "EMA Columns",
        "1st Standardization Method",
        "2nd Standardization Method",
        "Train data Shape", 
        "Test Data Shape", 
        
    ],
    "Details": [
        col_dependent,  # Dependent variable
        ', '.join(col_independent),  # independent variables
        f"{method}, Outlier treatement column : {col_dep}", # Outlier methods
        f"Lower limit ==== {lower_limit}, Number of Lower Outlier ==== {loc_lower_outlier.notna().sum()},Lower Limit Location ==== {loc_lower_outlier.to_dict()},Upper Limit ==== {upper_limit},Number of total upper outlier ==== {loc_upper_outlier.notna().sum()},Upper Limit Location ==== {loc_upper_outlier.to_dict()}",
        f"5th Percentile ==== {Q1}, 95th Percentile ==== {globals().get('Q4',)}",  # 5th and 95th Percentiole
        f"Log Column: {col_log}_Log(10), Lag Value: {lag_cols}", # Log and Lag column
        f"EMA Columns: {ma_cols}", # Moving Average Columns
        f"SMA Columns: {sma_cols}", # Exponential Moving Average Columns
        method_std, # 1st Standarize method
        method_std_2nd, # 1st Standarize method
        df_itteration.shape,  #Train Data shape
        df0.shape # Test Data shape
    ]
}

# Create the DataFrame
df_info = pd.DataFrame(data)

# Display the DataFrame
df_info


In [ ]:
# # Function to calculate VIF
# def calculate_vif(X):
#     vif_data = pd.DataFrame()
#     vif_data["Feature"] = X.columns
#     try:
#         vif_data["VIF"] = [VIF(X.values, i) for i in range(X.shape[1])]
#     except Exception as e:
#         print("Error calculating VIF:", e)
#         vif_data["VIF"] = np.nan
#     return vif_data

# # Function to calculate VIF efficiently
# def calculate_vif_matrix(X):
#     """Compute VIF in a vectorized manner instead of looping."""
#     vif = np.linalg.inv(X.corr().values).diagonal()
#     return vif



# # Generate all combinations of independent variables
# all_vars = df_itteration.columns.drop(Y1)
# combinations = []

# for r in range(comb_lower, comb_upper):
#     combinations.extend(itertools.combinations(all_vars, r))

# print("Total Combination", len(combinations))


# # Initialize lists to store results and regression summaries
# results = []
# regression_summaries = []

# # Iterate over all combinations
# for combo in tqdm(combinations, desc="Processing Time"):
#     X = df_itteration[list(combo)]
#     X_test = df0[list(combo)]
#     X = sm.add_constant(X)  # Add constant for intercept term
#     combo = list(X.columns)
#     X_test.insert(0, 'const', 1)

#     # Calculate VIF
#     # vif = [VIF(X.values, i) for i in range(X.shape[1])]

#     # **Vectorized VIF Calculation**
#     vif = calculate_vif_matrix(X)
    

#     if max(vif) <= vif_limit:
#         model = sm.OLS(df_itteration[Y1], X).fit()
#         if (model.pvalues <= p_val).all():  # Check if all p-values are <= 0.05
#             y_pred = model.predict(X)
#             r2 = round(r2_score(df_itteration[Y1], y_pred), 4)
#             n = len(df_itteration)
#             k = len(combo)
#             adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - k - 1))
#             adj_r2 = round(adj_r2, 4)

#             # Calculate estimates
#             lower_estimates = (X_test.mul(model.conf_int()[0])).sum(axis=1)
#             predictions = (X_test.mul(model.params)).sum(axis=1)
#             upper_estimates = (X_test.mul(model.conf_int()[1])).sum(axis=1)

#             # Filter conditions for estimates
#             if ((lower_estimates >= 0).all() and (predictions >= 0).all() and
#                 (upper_estimates >= 0).all() and (lower_estimates < predictions).all() and
#                 (predictions < upper_estimates).all()):
#                 sum_estimates = (np.round(lower_estimates.values + predictions.values + upper_estimates.values, 4))[0]
#                 if adj_r2 >= adj_r2_num:
#                     dw_statistic = round(durbin_watson(model.resid), 4)
#                     if dw_lower <= dw_statistic <= dw_upper:
#                         bp_test = het_breuschpagan(model.resid, X)
#                         bp_pvalue = round(bp_test[1], 4)

#                         # Capture regression summary as text
#                         reg_summary = model.summary2().as_text()

#                         # Append results
#                         results.append({
#                             'Variables': combo,
#                             'Coefficients': np.round(model.params.values, 4),
#                             'Lower Estimate': np.round(lower_estimates.values, 4)[0],
#                             'Estimates': np.round(predictions.values, 4)[0],
#                             'Upper Estimate': np.round(upper_estimates.values, 4)[0],
#                             'Sum of Estimates': sum_estimates,
#                             'R-squared': np.round(r2, 4),
#                             'Adjusted R-squared': np.round(adj_r2, 4),
#                             'P-values': np.round(model.pvalues.values, 4),
#                             'VIF': np.round(vif, 4),
#                             'Durbin-Watson': np.round(dw_statistic, 4),
#                             'Breusch-Pagan P Value': bp_pvalue,
#                             'Mean Residuals': model.resid.mean()
#                         })

#                         # Append regression summary to a separate list
#                         regression_summaries.append({
#                             'Variables': combo,
#                             'Regression Summary': reg_summary
#                         })

# # Create DataFrame from results
# results_df = pd.DataFrame(results)

# # Create a separate DataFrame for regression summaries
# regression_summary_df = pd.DataFrame(regression_summaries)

# # Function to check if row follows the correct direction
# def check_direction(variables, coefficients):
#     for var, coeff in zip(variables, coefficients):
#         if var in desired_directions:
#             if (coeff > 0 and desired_directions[var] == 'Negative') or \
#                (coeff < 0 and desired_directions[var] == 'Positive'):
#                 return False
#     return True

# print("Result_df Shape", results_df.shape)

# # Filter and sort results
# try:
#     results_df.sort_values(by=['Adjusted R-squared', 'Sum of Estimates', 'Durbin-Watson'], ascending=[False, True, True], inplace=True)
#     results_df_filtered = results_df[results_df.progress_apply(
#         lambda x: check_direction(x['Variables'], x['Coefficients']), axis=1)]
#     print("Final Combination Shape:", results_df_filtered.shape)

#     # Filter regression summaries based on the filtered results
#     filtered_variables = results_df_filtered['Variables'].tolist()
#     regression_summary_df_filtered = regression_summary_df[
#         regression_summary_df['Variables'].apply(lambda x: x in filtered_variables)
#     ]


#     # Align indices with the order in results_df_filtered
#     regression_summary_df_filtered = regression_summary_df_filtered.loc[results_df_filtered.index]
    
# except Exception as e:
#     print("Error during filtering:", e)

# # Display filtered regression summaries
# print("Filtered Regression Summaries DataFrame:",regression_summary_df_filtered.shape)


# Processing Models

In [ ]:

# # Function to calculate VIF efficiently
# def calculate_vif_matrix(X):
#     """Compute VIF in a vectorized manner instead of looping."""
#     vif = np.linalg.inv(X.corr().values).diagonal()
#     return vif

# # # Feature selection before combination generation (Optional)
# # corr_matrix = df_itteration.corr().abs()
# # upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
# # high_corr_features = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]
# # df_itteration = df_itteration.drop(columns=high_corr_features)  # Remove highly correlated features

# # Generate a limited number of combinations
# all_vars = df_itteration.columns.drop(Y1)
# combinations = []

# # **LIMIT THE MAX COMBINATIONS**
# # max_combinations = 10000  # Adjust based on dataset size
# for r in range(comb_lower, comb_upper):
#     new_combinations = list(itertools.combinations(all_vars, r))
#     # if len(combinations) + len(new_combinations) > max_combinations:
#     #     combinations.extend(new_combinations[: max_combinations - len(combinations)])
#     #     break
#     combinations.extend(new_combinations)

# print("Total Combinations After Limiting:", len(combinations))

# # Processing function
# def process_combination(combo):
#     X = df_itteration[list(combo)]
#     X_test = df0[list(combo)]
#     X = sm.add_constant(X)  # Add constant for intercept term
#     combo = list(X.columns)

#     X_test.insert(0, 'const', 1)

#     # **Vectorized VIF Calculation**
#     vif = calculate_vif_matrix(X)
#     if np.max(vif) > vif_limit:
#         return None  # Skip combinations with high VIF
    
#     elif max(vif) <= vif_limit:

#         # Fit OLS Model
#         model = sm.OLS(df_itteration[Y1], X).fit()

#         # **P-value Filtering in Bulk**
#         if np.any(model.pvalues > p_val):  # If any p-value > threshold, skip
#             return None

#         # Compute Adjusted R²
#         y_pred = model.predict(X)
#         r2 = r2_score(df_itteration[Y1], y_pred)
#         n, k = X.shape
#         adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - k - 1))

#         # Compute Estimates
#         lower_estimates = np.dot(X_test, model.conf_int().iloc[:, 0])
#         predictions = np.dot(X_test, model.params)
#         upper_estimates = np.dot(X_test, model.conf_int().iloc[:, 1])

#         # **Ensure Estimates Follow Constraints**
#         if not (np.all(lower_estimates >= 0) and np.all(predictions >= 0) and np.all(upper_estimates >= 0)):
#             return None

#         sum_estimates = round(np.sum(lower_estimates + predictions + upper_estimates), 4)
        
#         if adj_r2 < adj_r2_num:
#             return None  # Skip models with low Adjusted R²

#         # Durbin-Watson Test
#         dw_statistic = sm.stats.stattools.durbin_watson(model.resid)
#         if not (dw_lower <= dw_statistic <= dw_upper):
#             return None

#         # Breusch-Pagan Test
#         bp_test = het_breuschpagan(model.resid, X)
#         bp_pvalue = round(bp_test[1], 4)

#         # Store Results
#         result = {
#             'Variables': combo,
#             'Coefficients': model.params.values.round(4),
#             'Lower Estimate': lower_estimates.round(4)[0],
#             'Estimates': predictions.round(4)[0],
#             'Upper Estimate': upper_estimates.round(4)[0],
#             'Sum of Estimates': sum_estimates,
#             'R-squared': round(r2, 4),
#             'Adjusted R-squared': round(adj_r2, 4),
#             'P-values': model.pvalues.values.round(4),
#             'VIF': vif.round(4),
#             'Durbin-Watson': round(dw_statistic, 4),
#             'Breusch-Pagan P Value': bp_pvalue,
#             'Mean Residuals': model.resid.mean()
#         }

#         regression_summary = {
#             'Variables': combo,
#             'Regression Summary': model.summary2().as_text()
#         }

#         return result, regression_summary

# # **Parallel Processing**
# num_cores = -1  # Use all available cores
# processed_results = Parallel(n_jobs=num_cores)(
#     delayed(process_combination)(combo) for combo in tqdm(combinations, desc="Processing Models")
# )

# # **Filter None Values**
# processed_results = [res for res in processed_results if res is not None]

# if not processed_results:
#     print("No combination found")

# else:
#     results, regression_summaries = zip(*processed_results)

#     # Create DataFrames
#     results_df = pd.DataFrame(results)
#     regression_summary_df = pd.DataFrame(regression_summaries)

#     # **Vectorized Filtering Instead of Loop**
#     desired_directions_check = np.vectorize(lambda var, coeff: not (
#         (var in desired_directions and coeff > 0 and desired_directions[var] == 'Negative') or
#         (var in desired_directions and coeff < 0 and desired_directions[var] == 'Positive')
#     ))

#     valid_indices = [
#         i for i in range(len(results_df))
#         if desired_directions_check(results_df.iloc[i]['Variables'], results_df.iloc[i]['Coefficients']).all()
#     ]
#     results_df_filtered = results_df.iloc[valid_indices]
#     regression_summary_df_filtered = regression_summary_df.iloc[valid_indices]

#     # **Sort Results Efficiently**
#     results_df_filtered.sort_values(
#         by=['Adjusted R-squared', 'Sum of Estimates', 'Durbin-Watson'],
#         ascending=[False, True, True],
#         inplace=True
#     )

#     # Final Output
#     print("Final Combination Shape:", results_df_filtered.shape)
#     print("Filtered Regression Summaries DataFrame:", regression_summary_df_filtered.shape)


In [ ]:
# Function to calculate VIF
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    try:
        vif_data["VIF"] = [VIF(X.values, i) for i in range(X.shape[1])]
    except Exception as e:
        print("Error calculating VIF:", e)
        vif_data["VIF"] = np.nan
    return vif_data


# Generate all combinations of independent variables
all_vars = df_itteration.columns.drop(Y1)
combinations = []

for r in range(comb_lower, comb_upper):
    combinations.extend(itertools.combinations(all_vars, r))

print("Total Combination", len(combinations))


# Initialize lists to store results and regression summaries
results = []
regression_summaries = []

# Iterate over all combinations
for combo in tqdm(combinations, desc="Processing Time"):
    X = df_itteration[list(combo)]
    X_test = df0[list(combo)]
    X = sm.add_constant(X)  # Add constant for intercept term
    combo = list(X.columns)
    X_test.insert(0, 'const', 1)

    # Calculate VIF
    vif = [VIF(X.values, i) for i in range(X.shape[1])]

    if max(vif) <= vif_limit:
        model = sm.OLS(df_itteration[Y1], X).fit()
        if (model.pvalues <= p_val).all():  # Check if all p-values are <= 0.05
            y_pred = model.predict(X)
            r2 = round(r2_score(df_itteration[Y1], y_pred), 4)
            n = len(df_itteration)
            k = len(combo)
            adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - k - 1))
            adj_r2 = round(adj_r2, 4)

            # Calculate estimates
            lower_estimates = (X_test.mul(model.conf_int()[0])).sum(axis=1)
            predictions = (X_test.mul(model.params)).sum(axis=1)
            upper_estimates = (X_test.mul(model.conf_int()[1])).sum(axis=1)

            # Filter conditions for estimates
            if ((lower_estimates >= 0).all() and (predictions >= 0).all() and
                (upper_estimates >= 0).all() and (lower_estimates < predictions).all() and
                (predictions < upper_estimates).all()):
                sum_estimates = (np.round(lower_estimates.values + predictions.values + upper_estimates.values, 4))[0]
                if adj_r2 >= adj_r2_num:
                    dw_statistic = round(durbin_watson(model.resid), 4)
                    if dw_lower <= dw_statistic <= dw_upper:
                        bp_test = het_breuschpagan(model.resid, X)
                        bp_pvalue = round(bp_test[1], 4)

                        # Capture regression summary as text
                        reg_summary = model.summary2().as_text()

                        # Append results
                        results.append({
                            'Variables': combo,
                            'Coefficients': np.round(model.params.values, 4),
                            'Lower Estimate': np.round(lower_estimates.values, 4)[0],
                            'Estimates': np.round(predictions.values, 4)[0],
                            'Upper Estimate': np.round(upper_estimates.values, 4)[0],
                            'Sum of Estimates': sum_estimates,
                            'R-squared': np.round(r2, 4),
                            'Adjusted R-squared': np.round(adj_r2, 4),
                            'P-values': np.round(model.pvalues.values, 4),
                            'VIF': np.round(vif, 4),
                            'Durbin-Watson': np.round(dw_statistic, 4),
                            'Breusch-Pagan P Value': bp_pvalue,
                            'Mean Residuals': model.resid.mean()
                        })

                        # Append regression summary to a separate list
                        regression_summaries.append({
                            'Variables': combo,
                            'Regression Summary': reg_summary
                        })

# Create DataFrame from results
results_df = pd.DataFrame(results)

# Create a separate DataFrame for regression summaries
regression_summary_df = pd.DataFrame(regression_summaries)

# Function to check if row follows the correct direction
def check_direction(variables, coefficients):
    for var, coeff in zip(variables, coefficients):
        if var in desired_directions:
            if (coeff > 0 and desired_directions[var] == 'Negative') or \
               (coeff < 0 and desired_directions[var] == 'Positive'):
                return False
    return True

print("Result_df Shape", results_df.shape)

# Filter and sort results
try:
    results_df.sort_values(by=['Adjusted R-squared', 'Sum of Estimates', 'Durbin-Watson'], ascending=[False, True, True], inplace=True)
    results_df_filtered = results_df[results_df.progress_apply(
        lambda x: check_direction(x['Variables'], x['Coefficients']), axis=1)]
    print("Final Combination Shape:", results_df_filtered.shape)

    # Filter regression summaries based on the filtered results
    filtered_variables = results_df_filtered['Variables'].tolist()
    regression_summary_df_filtered = regression_summary_df[
        regression_summary_df['Variables'].apply(lambda x: x in filtered_variables)
    ]


    # Align indices with the order in results_df_filtered
    regression_summary_df_filtered = regression_summary_df_filtered.loc[results_df_filtered.index]
    
except Exception as e:
    print("Error during filtering:", e)

# Display filtered regression summaries
print("Filtered Regression Summaries DataFrame:",regression_summary_df_filtered.shape)


In [ ]:
# len(new_combinations)

In [ ]:
results_df_filtered.head(4)

# Export

In [ ]:
with pd.ExcelWriter(rf"{Folder_path}\{name} Possible All Combination log.xlsx",engine='xlsxwriter') as want:
    df_info.to_excel(want,sheet_name="Inputs Section",index=False)
    results_df_filtered.to_excel(want,sheet_name="Final Combination",index=False)
    regression_summary_df_filtered.to_excel(want,sheet_name="Regression Summary",index=False)
    results_df.to_excel(want,sheet_name="Comb WO Correct direction",index=False)
    summary_df.to_excel(want,sheet_name="Mean Std",index=False)
    model_data.to_excel(want,sheet_name="Model Data",index=False)
    
    

In [ ]:
# import pandas as pd

# df1 = pd.DataFrame({'A': [1, 2, 3,5,7]})
# df2 = pd.DataFrame({'B': [4, 5, 6]})

# # Concatenating column-wise
# result = pd.concat([df2,df1], axis=1)

# result
